In [4]:
import re
import numpy as np
import pandas as pd
import spacy
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
from pprint import pprint

In [5]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [9]:
df = pd.read_csv('../data/raw/tripadvisor.csv')

In [11]:
df.head()

,rating,review,title
0,50,What an experience. The hotel is centrally loc...,Beatiful Hotel
1,50,This hotel takes you back in time the best way...,Old world magic
2,50,Fantastic hotel in the heart of Antigua! The l...,Absolutely Wonderful
3,50,We had a dinner at this restaurant and had a g...,Restaurant at this hotel is great!
4,50,Loved every moment at this gorgeous hotel from...,exquisite location to explore Antigua from


In [12]:
df.loc[0,'review']

'What an experience. The hotel is centrally located in the heart of Antigua and one minute walk to the famous fountain, all shops, restaurants and all attractions. The rooms are very big, beautifully decorated and well maintained. The team are very well trained, very helpful and attentive. I would hoghly recommend this beautiful hotel.'

In [13]:
# Convertir a una lista
data = df.review.values.tolist()
pprint(data[:1])

['What an experience. The hotel is centrally located in the heart of Antigua '
 'and one minute walk to the famous fountain, all shops, restaurants and all '
 'attractions. The rooms are very big, beautifully decorated and well '
 'maintained. The team are very well trained, very helpful and attentive. I '
 'would hoghly recommend this beautiful hotel.']


In [15]:
def sent_to_words(sentences):
    for sentence in sentences:
    # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True elimina la puntuación

data_words = list(sent_to_words(data))

print(data_words[:1])

[['what', 'an', 'experience', 'the', 'hotel', 'is', 'centrally', 'located', 'in', 'the', 'heart', 'of', 'antigua', 'and', 'one', 'minute', 'walk', 'to', 'the', 'famous', 'fountain', 'all', 'shops', 'restaurants', 'and', 'all', 'attractions', 'the', 'rooms', 'are', 'very', 'big', 'beautifully', 'decorated', 'and', 'well', 'maintained', 'the', 'team', 'are', 'very', 'well', 'trained', 'very', 'helpful', 'and', 'attentive', 'would', 'hoghly', 'recommend', 'this', 'beautiful', 'hotel']]


In [16]:
# Construimos modelos de bigrams y trigrams
# https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Aplicamos el conjunto de bigrams/trigrams a nuestros documentos
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [17]:
print(bigram_mod[data_words[0]])
print(trigram_mod[bigram_mod[data_words[0]]])

['what', 'an', 'experience', 'the', 'hotel', 'is', 'centrally_located', 'in', 'the', 'heart', 'of', 'antigua', 'and', 'one', 'minute', 'walk', 'to', 'the', 'famous', 'fountain', 'all', 'shops', 'restaurants', 'and', 'all', 'attractions', 'the', 'rooms', 'are', 'very', 'big', 'beautifully_decorated', 'and', 'well', 'maintained', 'the', 'team', 'are', 'very', 'well', 'trained', 'very', 'helpful', 'and', 'attentive', 'would', 'hoghly', 'recommend', 'this', 'beautiful', 'hotel']
['what', 'an', 'experience', 'the', 'hotel', 'is', 'centrally_located', 'in', 'the', 'heart', 'of', 'antigua', 'and', 'one', 'minute', 'walk', 'to', 'the', 'famous', 'fountain', 'all', 'shops', 'restaurants', 'and', 'all', 'attractions', 'the', 'rooms', 'are', 'very', 'big', 'beautifully_decorated', 'and', 'well', 'maintained', 'the', 'team', 'are', 'very', 'well', 'trained', 'very', 'helpful', 'and', 'attentive', 'would', 'hoghly', 'recommend', 'this', 'beautiful', 'hotel']


In [18]:
# Eliminar stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Hacer bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Hacer trigrams
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Lematización basada en el modelo de POS de Spacy
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Eliminamos stopwords
data_words_nostops = remove_stopwords(data_words)

In [20]:
# Formamos bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [21]:
# python3 -m spacy download en_core_web_lg
# Inicializamos el modelo 'en_core_web_lg' con las componentes de POS únicamente
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

# Lematizamos preservando únicamente noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['experience', 'hotel', 'heart', 'minute', 'walk', 'famous', 'fountain', 'shop', 'restaurant', 'attraction', 'room', 'big', 'well', 'maintain', 'team', 'well', 'train', 'helpful', 'attentive', 'would', 'hoghly', 'recommend', 'beautiful', 'hotel']]


In [22]:
# Creamos diccionario
id2word = corpora.Dictionary(data_lemmatized)

In [23]:
for key, value in id2word.items():
    print(key, value)

0 attentive
1 attraction
2 beautiful
3 big
4 experience
5 famous
6 fountain
7 heart
8 helpful
9 hoghly
10 hotel
11 maintain
12 minute
13 recommend
14 restaurant
15 room
16 shop
17 team
18 train
19 walk
20 well
21 would
22 back
23 be
24 can
25 comfortable
26 enough
27 entire
28 feel
29 garden
30 good
31 gorgeous
32 make
33 peaceful
34 possible
35 say
36 stay
37 take
38 thing
39 time
40 way
41 area
42 bath
43 bathroom
44 bedroom
45 book
46 clean
47 couch
48 courtyard
49 curtain
50 door
51 eye
52 face
53 fantastic
54 first
55 floor
56 funny
57 furniture
58 give
59 go
60 highlight
61 huge
62 immaculately
63 incredible
64 inside
65 instrumental
66 leave
67 level
68 location
69 lounge
70 music
71 negative
72 open
73 outside
74 people
75 plenty
76 relax
77 right
78 separate
79 shower
80 sound
81 standard
82 table
83 tub
84 vintage
85 window
86 ambiance
87 company
88 dinner
89 end
90 enjoy
91 food
92 friend
93 great
94 highly
95 new
96 place
97 reserve
98 service
99 years_eve
100 absolutely
10

1157 destination
1158 efficiency
1159 honestly
1160 reluctant
1161 taking
1162 bottle
1163 consist
1164 daily
1165 fridge
1166 hiking
1167 maid
1168 map
1169 orientate
1170 upon_arrival
1171 firework
1172 overhead
1173 smooth
1174 surprise
1175 traditional
1176 trouble
1177 cream
1178 example
1179 figure
1180 frijol
1181 justice
1182 mum
1183 plantain
1184 post
1185 promptly
1186 refritos
1187 sour
1188 lounge_chair
1189 remember
1190 call
1191 cave
1192 cooking
1193 depend
1194 disappointed
1195 dusty
1196 duty
1197 grow
1198 guy
1199 lamp
1200 mold
1201 neglect
1202 obviously
1203 phone
1204 accurate
1205 additional
1206 arrival
1207 base
1208 construction
1209 email
1210 inquiry
1211 partially
1212 substantial
1213 memorable
1214 mistake
1215 organise
1216 packed
1217 ceiling_fan
1218 interior
1219 portion
1220 score
1221 sink
1222 hospitality
1223 poolside
1224 show
1225 thoroughly_enjoye
1226 umbrella
1227 d
1228 notice
1229 wine
1230 money
1231 accordingly
1232 cooler
1233 frill


2393 page
2394 pitcher
2395 regularly
2396 brew
2397 indigenous
2398 livable
2399 there
2400 weekly
2401 whip
2402 dustpan
2403 expise
2404 sluse
2405 stove
2406 thrill
2407 unlocked
2408 unsettling
2409 villa
2410 apology
2411 humidity
2412 smirk
2413 potential
2414 rare
2415 thief
2416 availabe
2417 insulate
2418 ratio
2419 storage
2420 weekday
2421 personally
2422 unreliable
2423 laugh
2424 expansive
2425 nail
2426 stable
2427 studio
2428 complication
2429 flag
2430 inclusive
2431 raisin
2432 resolve
2433 abandon
2434 basin
2435 black
2436 brown
2437 milk
2438 unkempt
2439 musty
2440 additionally
2441 generate
2442 cross
2443 stiff
2444 forever
2445 guarantee
2446 heap
2447 hostal
2448 madness
2449 resort
2450 fab
2451 flooring
2452 el_carmen
2453 camp
2454 insulation
2455 meh
2456 prevent
2457 smallest
2458 spotty
2459 battery
2460 ease
2461 glove
2462 headlight
2463 litre
2464 observe
2465 pole
2466 scarf
2467 ship
2468 shoe
2469 vegetarian
2470 whilst
2471 altitude
2472 appetite


3593 spin
3594 superbly
3595 hammok
3596 watch_movie
3597 cnd
3598 beatiful
3599 afterwards
3600 relation
3601 dump
3602 fashion
3603 conference
3604 singe
3605 dip
3606 gripe
3607 sunbathe
3608 absence
3609 okay
3610 behind
3611 exhaustive
3612 explosive
3613 fabulously
3614 passenger
3615 upscale
3616 veranda
3617 yesterday
3618 seasoned
3619 wanted
3620 amazed
3621 breathe
3622 cardboard
3623 mouldy
3624 quietly
3625 spiky
3626 towell
3627 friday
3628 prebook
3629 appreciative
3630 sleepy
3631 willingness
3632 extravagant
3633 wise
3634 purposely
3635 ripe
3636 tablespoon
3637 feather
3638 mom
3639 taken
3640 unrelated
3641 gap
3642 properly
3643 smoking
3644 toothbrush
3645 marked
3646 groomed
3647 dishware
3648 lodge
3649 pots_pan
3650 rave
3651 hebrew
3652 cheaper
3653 mint
3654 presentation
3655 spicy
3656 happy_hour
3657 inescapable
3658 sake
3659 stress
3660 filtered_water
3661 hygiene
3662 cane
3663 frame
3664 splinter
3665 weed
3666 marketplace
3667 continued
3668 creaky
366

4773 kick
4774 random
4775 squeak
4776 stink
4777 clearing
4778 occasionally
4779 queensize
4780 tardy
4781 merit
4782 alleyway
4783 esthela
4784 affordability
4785 buck
4786 greenhouse
4787 bagel
4788 dehumidifier
4789 deposit_box
4790 foe
4791 redecoration
4792 committed
4793 cloth
4794 stall
4795 weave
4796 boring
4797 previous_reviewer
4798 dingy
4799 reheat
4800 crescent
4801 ferry
4802 slowly
4803 hitch
4804 manicured
4805 opened_onto
4806 protect
4807 quarter
4808 heartly
4809 wonderfull
4810 dificult
4811 receipt
4812 sum
4813 butchering
4814 dishearten
4815 meson
4816 body
4817 coming
4818 discard
4819 going
4820 qualm
4821 boom
4822 inevitably
4823 orchestrate
4824 agencey
4825 concious
4826 summon
4827 stuffed
4828 torch
4829 bid
4830 planet
4831 chatting
4832 output
4833 disconcerting
4834 dig
4835 straighten
4836 understatement
4837 eally
4838 rearrange
4839 luxurio
4840 glam
4841 glamping
4842 desirable
4843 infuriating
4844 karaoke
4845 unimpressed
4846 fingertip
4847 pr

...

7089 spigot
7090 emotion
7091 rightfully
7092 altho
7093 blooms
7094 flutter
7095 hostal_la
7096 sliver
7097 cupcake
7098 recieve
7099 purfied
7100 piping
7101 chase
7102 attendance
7103 delicately
7104 luscious
7105 transpiration
7106 enchanting
7107 detract
7108 vegetation
7109 shuffling
7110 bearly
7111 culdnt
7112 impolite
7113 webpage
7114 mislead
7115 pinch
7116 unclear
7117 recharge
7118 adeptly
7119 partake
7120 prevalent
7121 unreserved
7122 beautfiul
7123 imposible
7124 magic
7125 tedious
7126 saver
7127 mainteined
7128 melt
7129 carefree
7130 santa_ine
7131 coup
7132 scenario
7133 adn
7134 dicember
7135 torture
7136 acct
7137 spit
7138 dart
7139 gaff
7140 wicked
7141 erratic
7142 prepay
7143 perimeter
7144 sharply
7145 discomforting
7146 incude
7147 mantained
7148 restauraunt
7149 animate
7150 quilt
7151 stones_throw
7152 familys
7153 tireless
7154 privat
7155 casa_luna
7156 disregard
7157 energize
7158 fierce
7159 ought
7160 thunderstorm
7161 revisit
7162 doublebooked
7163 

8239 hiatal
8240 brilliantly
8241 grip
8242 added_bonu
8243 elote
8244 sneak
8245 bespoke
8246 bail
8247 misheard
8248 salon
8249 cleanup
8250 remedie
8251 guestroom
8252 passage
8253 haggle
8254 drumbeat
8255 button
8256 vegitation
8257 perfectky
8258 sanctum
8259 nomber
8260 affix
8261 facade
8262 fullsize
8263 letter
8264 underutilise
8265 hamburger
8266 compe
8267 sweetly
8268 correspondance
8269 personalized
8270 immense
8271 withn
8272 sauteed
8273 staple
8274 toolkit
8275 airplane
8276 par
8277 slim
8278 digital
8279 hover
8280 rosy
8281 threaten
8282 imho
8283 peril
8284 popularly
8285 executive
8286 recycling
8287 enjoyably
8288 chimaltenago
8289 niece
8290 sangria
8291 dome
8292 kno
8293 nightime
8294 driveway
8295 surly
8296 becuase
8297 fame
8298 famed
8299 bud
8300 coquettishly
8301 dearth
8302 explosivity
8303 index
8304 lament
8305 melodious
8306 sea
8307 snugly
8308 yoga_clas
8309 cabana
8310 staid
8311 existent
8312 frenzy
8313 climatic
8314 uneasy
8315 unusable
8316 u

9513 chest
9514 converstaion
9515 hoping
9516 insure
9517 marginal
9518 mitigate
9519 unscreened
9520 island
9521 pilar
9522 apalled
9523 autumn
9524 fungus
9525 journalist
9526 norwegian
9527 sound_proof
9528 minus
9529 comparative
9530 fernados
9531 damaged
9532 toal
9533 nectar
9534 valueformoney
9535 campanie
9536 coeliac
9537 furry
9538 potholder
9539 seamlessly
9540 defintly
9541 skirting
9542 valance
9543 incide
9544 jungle
9545 stained
9546 deg
9547 saint
9548 antiquity
9549 exagerate
9550 preclude
9551 marimba_music
9552 bandaid
9553 carnivore
9554 washrooms
9555 uncle
9556 wedding_anniversary
9557 elementary
9558 casita
9559 mist
9560 foody
9561 pleacent
9562 remmembered
9563 replicate
9564 consumer
9565 personel
9566 consumption
9567 abhor
9568 trace
9569 kittie
9570 belch
9571 audiobook
9572 ace
9573 newest
9574 contiously
9575 luxary
9576 plumed
9577 jean
9578 organically
9579 dbl
9580 neurotically
9581 repacke
9582 slippery
9583 warmish
9584 sometype
9585 versa
9586 chari

10614 compelling
10615 usher
10616 incarnate
10617 scrabble
10618 acorn
10619 patter
10620 pitter
10621 immensly
10622 slime
10623 itallian
10624 thai
10625 carpark
10626 disponible
10627 inthe
10628 communion
10629 openning
10630 pave
10631 absolutey
10632 cirilo
10633 restorative
10634 restauarant
10635 outsatnde
10636 carbohydrate
10637 commonarea
10638 alleviate
10639 anxiety
10640 inconsistently
10641 accountability
10642 lirio
10643 trigger
10644 airless
10645 aesthete
10646 mil
10647 achievement
10648 bogus
10649 tripe
10650 involoved
10651 donkey
10652 ehxilarate
10653 tarantula
10654 quid
10655 snobbishness
10656 underlie
10657 rested
10658 usualy
10659 teodora
10660 escargot
10661 prominently
10662 fullly
10663 origin
10664 rikis
10665 exposure
10666 solved
10667 grading
10668 gazebo
10669 paranoid
10670 amble
10671 birder
10672 vibrancy
10673 bernaise
10674 orquideas
10675 pluse
10676 impeccible
10677 queirie
10678 transplant
10679 disjoint
10680 repaint
10681 bicarb
10682 d

11878 reproduce
11879 revitalize
11880 destruction
11881 inherent
11882 quake
11883 accessing
11884 evry
11885 difficoult
11886 classroom
11887 literacy
11888 melody
11889 camarone
11890 proactiveness
11891 comming
11892 weaver
11893 offputting
11894 scold
11895 bubblegum
11896 germ
11897 scan
11898 violet
11899 disapointing
11900 ment
11901 maoavilloso
11902 realmente
11903 foc
11904 confinement
11905 flick
11906 colonional
11907 untouched
11908 defend
11909 idiot
11910 poisening
11911 deagua
11912 innocuous
11913 looong
11914 vow
11915 conventioneer
11916 merinques
11917 assemble
11918 domincan
11919 provocative
11920 carpeted
11921 liturgical
11922 fiercely
11923 ravioli
11924 mutch
11925 creme
11926 fleece
11927 overcook
11928 paltry
11929 halways
11930 laught
11931 procede
11932 clash
11933 unremarkable
11934 arara
11935 guestrooms
11936 abosoluitely
11937 magestic
11938 signaling
11939 humane
11940 restarant
11941 intereting
11942 removte
11943 ameican
11944 consultant
11945 folo

In [24]:
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1)]]


In [26]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('attentive', 1),
  ('attraction', 1),
  ('beautiful', 1),
  ('big', 1),
  ('experience', 1),
  ('famous', 1),
  ('fountain', 1),
  ('heart', 1),
  ('helpful', 1),
  ('hoghly', 1),
  ('hotel', 2),
  ('maintain', 1),
  ('minute', 1),
  ('recommend', 1),
  ('restaurant', 1),
  ('room', 1),
  ('shop', 1),
  ('team', 1),
  ('train', 1),
  ('walk', 1),
  ('well', 2),
  ('would', 1)]]

## Latent Dirichlet Allocation

![Latent Dirichlet Allocation](https://miro.medium.com/max/1400/1*fCc0JT3W-1ViYyw0hJ7rdA.jpeg)

In [48]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [121]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
#lda_model.top_topics()
#pprint(corpus[:1][0])

[(0,
  '0.049*"suite" + 0.041*"site" + 0.039*"block" + 0.038*"charge" + '
  '0.033*"colonial" + 0.024*"live" + 0.023*"church" + 0.016*"original" + '
  '0.015*"outdoor" + 0.014*"luxurious"'),
 (1,
  '0.039*"fireplace" + 0.034*"monastery" + 0.029*"absolutely" + 0.027*"unique" '
  '+ 0.026*"gorgeous" + 0.024*"build" + 0.024*"must" + 0.023*"modern" + '
  '0.022*"fantastic" + 0.020*"convent"'),
 (2,
  '0.047*"night" + 0.025*"could" + 0.023*"get" + 0.021*"even" + 0.018*"find" + '
  '0.017*"first" + 0.015*"book" + 0.014*"go" + 0.014*"say" + 0.013*"give"'),
 (3,
  '0.056*"stay" + 0.053*"place" + 0.049*"good" + 0.049*"great" + 0.047*"staff" '
  '+ 0.027*"food" + 0.026*"nice" + 0.023*"really" + 0.022*"friendly" + '
  '0.018*"also"'),
 (4,
  '0.115*"hotel" + 0.035*"beautiful" + 0.030*"stay" + 0.026*"service" + '
  '0.021*"make" + 0.020*"ground" + 0.018*"restaurant" + 0.016*"museum" + '
  '0.015*"visit" + 0.015*"time"'),
 (5,
  '0.106*"ruin" + 0.054*"art" + 0.032*"surround" + 0.022*"fire" + '
  '0

In [90]:
doc_lda = lda_model[corpus[:1]]

In [125]:
topics = list()
for prob in doc_lda[0][1]:
    for topic in prob[1]:
        topics.append(topic)
moda = max(set(topics), key=topics.count)
topics = lda_model.print_topics()[moda]
pprint(topics)

(4,
 '0.115*"hotel" + 0.035*"beautiful" + 0.030*"stay" + 0.026*"service" + '
 '0.021*"make" + 0.020*"ground" + 0.018*"restaurant" + 0.016*"museum" + '
 '0.015*"visit" + 0.015*"time"')


In [126]:
#Perplejidad
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Score de coherencia
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.993208215295965

Coherence Score:  0.3915563571341127


In [128]:
# Visualizamos los temas
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\Daniel\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.164521 -0.035467       1        1  27.140459
4     -0.217091  0.171215       2        1  24.217150
3     -0.219902  0.120306       3        1  23.592545
2     -0.069640 -0.390649       4        1  15.281749
1      0.229117  0.100839       5        1   4.162450
0      0.217913  0.013513       6        1   2.978908
5      0.224124  0.020242       7        1   2.626737, topic_info=      Category          Freq       Term         Total  loglift  logprob
15     Default  18746.000000       room  18746.000000  30.0000  30.0000
10     Default  16685.000000      hotel  16685.000000  29.0000  29.0000
96     Default   7487.000000      place   7487.000000  28.0000  28.0000
93     Default   6873.000000      great   6873.000000  27.0000  27.0000
30     Default   6919.000000       good   6919.000000  26.0000  26.0000
...        ...           ...        ...           ...      ...      ...
1213    Topic7    111.893341  memorable    112.677383   3.6324  -4.9380
986     Topic7    110.038223      wi_fi    110.821640   3.6323  -4.9547
10339   Topic7    127.567703      macaw    128.476089   3.6323  -4.8069
386     Topic7    109.525833       calm    110.308914   3.6323  -4.9594
1947    Topic7    107.247589  concierge    108.038116   3.6321  -4.9804

[301 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
100       5  0.999137  absolutely
1243      3  0.998955      almost
258       1  0.256461        also
258       3  0.743383        also
292       2  0.022131      always
...     ...       ...         ...
21        3  0.629149       would
21        4  0.050201       would
1363      7  0.992320       write
291       6  0.993752       wrong
532       2  0.998920        year

[335 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 4, 3, 2, 1, 6])